In [1]:
using Distances

In [2]:
struct CVRPInstance
    cost::Array{Float64, 2}
    demand::Array{Int64}
    capacity::Int64
    K::Int64
end

In [3]:
function parse_vrp(fp::String)

    f = open(fp)
    text = readlines(f)
    close(f)

    k = 0
    capacity = 0
    instance_size = 0

    data_parse_flag = false
    parsing_demand = false
    parsing_edge_weights = false
    parsing_coords = false

    coords = []
    demand = []
    cost = []

    for line in text
        if occursin("NAME : ", line)
            k += parse(Int, match(r"k\d+", line).match[2:end])
        elseif occursin("DIMENSION", line)
            instance_size += parse(Int, match(r"\d+", line).match)
        elseif occursin("CAPACITY", line)
            capacity += parse(Int, match(r"\d+", line).match)
        elseif occursin("NODE_COORD_SECTION", line)
            data_parse_flag = true
            parsing_coords = true
            parsing_edge_weights = false
            parsing_demand = false
            continue
        elseif occursin("EDGE_WEIGHT_SECTION", line)
            data_parse_flag = true
            parsing_coords = false
            parsing_edge_weights = true
            parsing_demand = false
            continue
        elseif occursin("DEMAND_SECTION", line)
            data_parse_flag = true
            parsing_coords = false
            parsing_edge_weights = false
            parsing_demand = true
            continue
        elseif occursin("DEPOT_SECTION", line)
            data_parse_flag = false

        elseif occursin("EOF", line)
            break
        end

        if data_parse_flag
            if parsing_coords
                data = parse.(Float64, split(line))
                push!(coords, data)
            elseif parsing_edge_weights
                data = parse.(Float64, split(line))
                push!(cost, data)
            elseif parsing_demand
                data = parse.(Int, split(line))
                push!(demand, data)
            end
        end
    end

    if size(coords)[end] == 0
        println("Sorry not implemented")
    else
        coords = permutedims(hcat(coords...))[:,2:end];
        demand = permutedims(hcat(demand...))[:,2];
        cost = pairwise(Euclidean(), coords, dims=1)

        return CVRPInstance(cost, demand, capacity, k)
    end
end

parse_vrp (generic function with 1 method)

In [6]:
instance = parse_vrp("data/E-n76-k14.vrp")

CVRPInstance([0.0 25.45584412271571 … 20.0 3.0; 25.45584412271571 0.0 … 18.110770276274835 23.430749027719962; … ; 20.0 18.110770276274835 … 0.0 17.0; 3.0 23.430749027719962 … 17.0 0.0], [0, 18, 26, 11, 30, 21, 19, 15, 16, 29  …  37, 30, 10, 8, 11, 3, 1, 6, 10, 20], 100, 14)